In [38]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
GPU (s):
0.044843599000159884


2023-05-04 04:31:41.342147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 04:31:41.342325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 04:31:41.342437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 04:31:41.342599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 04:31:41.342715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [39]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [40]:
# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

In [41]:
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Load the CIFAR-100 dataset
data_path = 'cifar-100-python/'
train_data_100 = unpickle(data_path + 'train')[b'data']
train_data_100 = train_data_100.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data_100 = unpickle(data_path + 'test')[b'data']
test_data_100 = test_data_100.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels_100 = np.array(unpickle(data_path + 'train')[b'fine_labels'])
test_labels_100 = np.array(unpickle(data_path + 'test')[b'fine_labels'])

In [42]:
# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Print the number of features, classes, and training samples for CIFAR-100
print("\nCIFAR-100")
print("Number of features:", train_data_100.shape[1:])
print("Number of classes:", len(np.unique(train_labels_100)))
print("Number of training samples:", train_data_100.shape[0])

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000

CIFAR-100
Number of features: (32, 32, 3)
Number of classes: 100
Number of training samples: 50000


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
train_data_100, train_labels_100 = shuffle(train_data_100, train_labels_100, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)
test_data_100, test_labels_100 = shuffle(test_data_100, test_labels_100, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0
x_train_100 = train_data_100.astype('float32') / 255.0
x_test_100 = test_data_100.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)
y_train_100 = to_categorical(train_labels_100, num_classes=100)
y_test_100 = to_categorical(test_labels_100, num_classes=100)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

In [46]:
#Build ResNet-50 model
#base_model = ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
#predictions = Dense(10, activation='softmax')(x)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization

model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model_10.add(BatchNormalization())
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(BatchNormalization())
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(BatchNormalization())
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(BatchNormalization())
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(BatchNormalization())
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(BatchNormalization())
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(BatchNormalization())
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

#model_10 = Model(inputs=base_model.input, outputs=predictions)

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu.{epoch:02d}.h5', save_freq='epoch')

learning_rate = 0.001

# Define a learning rate schedule function
def lr_schedule(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.001
    if (epoch>1 and epoch<=3):
        learning_rate = 0.00075
    if (epoch>3 and epoch<=5):
        learning_rate = 0.0005
    if (epoch>5 and epoch<=7):
        learning_rate = 0.00025
    if (epoch>7 and epoch<=9):
        learning_rate = 0.000125
    if (epoch>9 and epoch<=11):
        learning_rate = 0.0000625
    if (epoch>11 and epoch<=13):
        learning_rate = 0.00003125
    if (epoch>13 and epoch<= 15):
        learning_rate = 0.00001575
    if (epoch>15):
        learning_rate = 0.000007375
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

#model_10.compile(optimizer=SGD(learning_rate=learning_rate, momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

In [47]:
# Model Fit
model_10.fit(it_train, steps_per_epoch=steps, epochs=400, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Epoch 1/400
781/781 [==============================] - 15s 18ms/step - loss: 2.1393 - accuracy: 0.2986 - val_loss: 1.4816 - val_accuracy: 0.4613
Epoch 2/400
781/781 [==============================] - 14s 18ms/step - loss: 1.6165 - accuracy: 0.4080 - val_loss: 1.4033 - val_accuracy: 0.4932
Epoch 3/400
781/781 [==============================] - 14s 18ms/step - loss: 1.4910 - accuracy: 0.4538 - val_loss: 1.3270 - val_accuracy: 0.5201
Epoch 4/400
781/781 [==============================] - 14s 18ms/step - loss: 1.4223 - accuracy: 0.4832 - val_loss: 1.3085 - val_accuracy: 0.5318
Epoch 5/400
781/781 [==============================] - 13s 17ms/step - loss: 1.3714 - accuracy: 0.5031 - val_loss: 1.4290 - val_accuracy: 0.4899
Epoch 6/400
781/781 [==============================] - 13s 17ms/step - loss: 1.3174 - accuracy: 0.5235 - val_loss: 1.3309 - val_accuracy: 0.5151
Epoch 7/400
781/781 [==============================] - 14s 18ms/step - loss: 1.2740 - accuracy: 0.5414 - val_loss: 1.2229 - val_ac

781/781 [==============================] - 15s 19ms/step - loss: 0.6708 - accuracy: 0.7675 - val_loss: 0.6327 - val_accuracy: 0.7783
Epoch 58/400
781/781 [==============================] - 15s 19ms/step - loss: 0.6677 - accuracy: 0.7685 - val_loss: 0.6639 - val_accuracy: 0.7660
Epoch 59/400
781/781 [==============================] - 13s 17ms/step - loss: 0.6591 - accuracy: 0.7733 - val_loss: 0.6768 - val_accuracy: 0.7638
Epoch 60/400
781/781 [==============================] - 14s 18ms/step - loss: 0.6527 - accuracy: 0.7731 - val_loss: 0.6441 - val_accuracy: 0.7774
Epoch 61/400
781/781 [==============================] - 15s 19ms/step - loss: 0.6554 - accuracy: 0.7737 - val_loss: 0.6181 - val_accuracy: 0.7847
Epoch 62/400
781/781 [==============================] - 14s 18ms/step - loss: 0.6448 - accuracy: 0.7776 - val_loss: 0.6407 - val_accuracy: 0.7763
Epoch 63/400
781/781 [==============================] - 15s 19ms/step - loss: 0.6402 - accuracy: 0.7797 - val_loss: 0.6773 - val_accuracy

781/781 [==============================] - 14s 18ms/step - loss: 0.5042 - accuracy: 0.8282 - val_loss: 0.4924 - val_accuracy: 0.8315
Epoch 114/400
781/781 [==============================] - 13s 17ms/step - loss: 0.5073 - accuracy: 0.8264 - val_loss: 0.4593 - val_accuracy: 0.8437
Epoch 115/400
781/781 [==============================] - 14s 17ms/step - loss: 0.5019 - accuracy: 0.8284 - val_loss: 0.5238 - val_accuracy: 0.8227
Epoch 116/400
781/781 [==============================] - 15s 20ms/step - loss: 0.5042 - accuracy: 0.8271 - val_loss: 0.4630 - val_accuracy: 0.8435
Epoch 117/400
781/781 [==============================] - 14s 18ms/step - loss: 0.4986 - accuracy: 0.8285 - val_loss: 0.4540 - val_accuracy: 0.8467
Epoch 118/400
781/781 [==============================] - 15s 19ms/step - loss: 0.4956 - accuracy: 0.8295 - val_loss: 0.5257 - val_accuracy: 0.8251
Epoch 119/400
781/781 [==============================] - 14s 18ms/step - loss: 0.4911 - accuracy: 0.8313 - val_loss: 0.5138 - val_ac

Epoch 169/400
781/781 [==============================] - 14s 18ms/step - loss: 0.4253 - accuracy: 0.8539 - val_loss: 0.4619 - val_accuracy: 0.8477
Epoch 170/400
781/781 [==============================] - 15s 19ms/step - loss: 0.4290 - accuracy: 0.8521 - val_loss: 0.4458 - val_accuracy: 0.8515
Epoch 171/400
781/781 [==============================] - 13s 17ms/step - loss: 0.4244 - accuracy: 0.8556 - val_loss: 0.4200 - val_accuracy: 0.8612
Epoch 172/400
781/781 [==============================] - 14s 17ms/step - loss: 0.4204 - accuracy: 0.8562 - val_loss: 0.4515 - val_accuracy: 0.8480
Epoch 173/400
781/781 [==============================] - 15s 19ms/step - loss: 0.4195 - accuracy: 0.8558 - val_loss: 0.4471 - val_accuracy: 0.8518
Epoch 174/400
781/781 [==============================] - 14s 18ms/step - loss: 0.4180 - accuracy: 0.8563 - val_loss: 0.4159 - val_accuracy: 0.8585
Epoch 175/400
781/781 [==============================] - 13s 17ms/step - loss: 0.4124 - accuracy: 0.8570 - val_loss: 0

Epoch 225/400
781/781 [==============================] - 15s 19ms/step - loss: 0.3679 - accuracy: 0.8726 - val_loss: 0.3936 - val_accuracy: 0.8688
Epoch 226/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3712 - accuracy: 0.8716 - val_loss: 0.3845 - val_accuracy: 0.8717
Epoch 227/400
781/781 [==============================] - 14s 17ms/step - loss: 0.3694 - accuracy: 0.8722 - val_loss: 0.3965 - val_accuracy: 0.8711
Epoch 228/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3720 - accuracy: 0.8739 - val_loss: 0.4082 - val_accuracy: 0.8644
Epoch 229/400
781/781 [==============================] - 14s 17ms/step - loss: 0.3701 - accuracy: 0.8726 - val_loss: 0.3687 - val_accuracy: 0.8778
Epoch 230/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3710 - accuracy: 0.8732 - val_loss: 0.3777 - val_accuracy: 0.8743
Epoch 231/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3699 - accuracy: 0.8736 - val_loss: 0

Epoch 281/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3376 - accuracy: 0.8834 - val_loss: 0.3787 - val_accuracy: 0.8765
Epoch 282/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3355 - accuracy: 0.8844 - val_loss: 0.3789 - val_accuracy: 0.8794
Epoch 283/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3394 - accuracy: 0.8840 - val_loss: 0.4032 - val_accuracy: 0.8684
Epoch 284/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3385 - accuracy: 0.8818 - val_loss: 0.3966 - val_accuracy: 0.8710
Epoch 285/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3384 - accuracy: 0.8848 - val_loss: 0.3689 - val_accuracy: 0.8799
Epoch 286/400
781/781 [==============================] - 13s 17ms/step - loss: 0.3339 - accuracy: 0.8844 - val_loss: 0.4050 - val_accuracy: 0.8670
Epoch 287/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3334 - accuracy: 0.8857 - val_loss: 0

Epoch 337/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3139 - accuracy: 0.8917 - val_loss: 0.3702 - val_accuracy: 0.8790
Epoch 338/400
781/781 [==============================] - 13s 17ms/step - loss: 0.3124 - accuracy: 0.8919 - val_loss: 0.3699 - val_accuracy: 0.8792
Epoch 339/400
781/781 [==============================] - 14s 17ms/step - loss: 0.3066 - accuracy: 0.8936 - val_loss: 0.3590 - val_accuracy: 0.8818
Epoch 340/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3144 - accuracy: 0.8921 - val_loss: 0.3386 - val_accuracy: 0.8890
Epoch 341/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3105 - accuracy: 0.8926 - val_loss: 0.3544 - val_accuracy: 0.8852
Epoch 342/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3095 - accuracy: 0.8940 - val_loss: 0.3618 - val_accuracy: 0.8829
Epoch 343/400
781/781 [==============================] - 14s 18ms/step - loss: 0.3066 - accuracy: 0.8936 - val_loss: 0

Epoch 393/400
781/781 [==============================] - 14s 18ms/step - loss: 0.2864 - accuracy: 0.9019 - val_loss: 0.3560 - val_accuracy: 0.8833
Epoch 394/400
781/781 [==============================] - 14s 18ms/step - loss: 0.2941 - accuracy: 0.8969 - val_loss: 0.3451 - val_accuracy: 0.8891
Epoch 395/400
781/781 [==============================] - 14s 18ms/step - loss: 0.2911 - accuracy: 0.8990 - val_loss: 0.3601 - val_accuracy: 0.8880
Epoch 396/400
781/781 [==============================] - 13s 17ms/step - loss: 0.2885 - accuracy: 0.8997 - val_loss: 0.3379 - val_accuracy: 0.8897
Epoch 397/400
781/781 [==============================] - 13s 17ms/step - loss: 0.2886 - accuracy: 0.8996 - val_loss: 0.3385 - val_accuracy: 0.8910
Epoch 398/400
781/781 [==============================] - 14s 18ms/step - loss: 0.2857 - accuracy: 0.9017 - val_loss: 0.3482 - val_accuracy: 0.8890
Epoch 399/400
781/781 [==============================] - 15s 19ms/step - loss: 0.2866 - accuracy: 0.9011 - val_loss: 0

In [36]:
# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.8586 - accuracy: 0.6994
Test accuracy: 0.699400007724762


In [22]:
# create data generator
datagen_100 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train_100 = datagen.flow(x_train_100, y_train_100, batch_size=64)
# fit model
steps = int(x_train_100.shape[0] / 64)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

model_100 = Sequential()
model_100.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model_100.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.2))
model_100.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.3))
model_100.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.4))
model_100.add(Flatten())
model_100.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_100.add(Dropout(0.5))
model_100.add(Dense(100, activation='softmax'))

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_100_gpu.{epoch:02d}.h5', save_freq='epoch')

learning_rate = 0.0001

# Define a learning rate schedule function
def lr_schedule(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.001
    if (epoch>130 and epoch<=200):
        learning_rate=0.0005
    if (epoch>200 and epoch<=250):
        learning_rate=0.00025
    if (epoch>250 and epoch<=300):
        learning_rate=0.000125
    if (epoch>300 and epoch<=350):
        learning_rate=0.0000625
    if (epoch>300 and epoch<=400):
        learning_rate=0.00003125
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_100.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

# Model Fit
#model_100.fit(it_train_100, steps_per_epoch=steps, epochs=400, validation_data=(x_test_100, y_test_100), callbacks=[checkpoint_callback])

In [9]:
from tensorflow.keras.models import load_model

# load the pre-saved model
model_100 = load_model('model_100_gpu.105.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_100.evaluate(x_test_100, y_test_100)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 1.9054 - accuracy: 0.4883
Test accuracy: 0.48829999566078186


In [23]:
learning_rate=0.0005

# Define a learning rate schedule function
def lr_schedule_new(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.0005
    if (epoch>50 and epoch<=100):
        learning_rate=0.00025
    if (epoch>100 and epoch<=150):
        learning_rate=0.000125
    if (epoch>150 and epoch<=200):
        learning_rate=0.0000625
    if (epoch>200 and epoch<=250):
        learning_rate=0.00003125
    if (epoch>300 and epoch<=400):
        learning_rate=0.000015625
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule_new)

model_100.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

# Model Fit
#model_100.fit(it_train_100, steps_per_epoch=steps, epochs=400, validation_data=(x_test_100, y_test_100), callbacks=[lr_scheduler,checkpoint_callback])

In [24]:
from tensorflow.keras.models import load_model

# load the pre-saved model
model_100 = load_model('model_100_gpu.400.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_100.evaluate(x_test_100, y_test_100)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 1.5624 - accuracy: 0.5890
Test accuracy: 0.5889999866485596
